In [114]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
import matplotlib.pyplot as plt
import numpy as np
import sys
import astropy.units as u
from astropy.io import ascii, fits
from astropy.modeling.blackbody import blackbody_lambda

sys.path.insert(0, '../')
from libra import Spot, Star

In [115]:
from glob import glob

paths = glob('/Users/bmmorris/git/freckles/data/lte*BT-Settl.spec.fits')

from astropy.io import fits

phoenix_wavelengths = fits.getdata(paths[0])['Wavelength']

temperatures = []
model_fluxes = []

for path in paths:
    hdu = fits.open(path)
    temperatures.append(hdu[1].header['PHXTEFF'])
    interp_flux = np.interp(wavelengths, hdu[1].data['Wavelength'], hdu[1].data['Flux'])
    model_fluxes.append(interp_flux)
    
model_fluxes = np.vstack(model_fluxes)
temperatures = np.array(temperatures)

In [116]:
V = 18.8
R = 16.6

def sort_transmittance(*args):
    sort = np.argsort(args[0])
    return lambda x: np.interp(x, args[0][sort]/1000, args[1][sort]/1000, left=0, right=0)

trans_v = sort_transmittance(*np.loadtxt('../libra/data/filters/Bessel_V-1.txt', unpack=True, delimiter='\t\t'))
trans_b = sort_transmittance(*np.loadtxt('../libra/data/filters/Bessel_B-1.txt', unpack=True, delimiter='\t\t'))
trans_r = sort_transmittance(*np.loadtxt('../libra/data/filters/Bessel_R-1.txt', unpack=True, delimiter='\t\t'))
trans_i = sort_transmittance(*np.loadtxt('../libra/data/filters/Bessel_I-1.txt', unpack=True, delimiter='\t\t'))


$$ V - R = - 2.5 \log_{10} \frac{\int_V F_\lambda T_{V,\lambda} \lambda d\lambda}{\int_R F_\lambda T_{R,\lambda} \lambda d\lambda} $$

In [121]:
def flux_to_b_minus_v(wavelength, flux):
    flux_v = np.trapz(flux * wavelength * trans_v(wavelength), wavelength)
    flux_b = np.trapz(flux * wavelength * trans_b(wavelength), wavelength)
    return - 2.5 * np.log10(flux_b / flux_v)

def flux_to_v_minus_r(wavelength, flux):
    flux_v = np.trapz(flux * wavelength * trans_v(wavelength), wavelength)
    flux_r = np.trapz(flux * wavelength * trans_r(wavelength), wavelength)
    return - 2.5 * np.log10(flux_v / flux_r)

def flux_to_v_minus_i(wavelength, flux):
    flux_v = np.trapz(flux * wavelength * trans_v(wavelength), wavelength)
    flux_i = np.trapz(flux * wavelength * trans_i(wavelength), wavelength)
    return - 2.5 * np.log10(flux_v / flux_i)

Mamajek: K8V: 4000 K

B-V = 1.356,  V-R = 0.843,  V-I = 1.632

In [124]:
k8v = model_fluxes[np.argmin(np.abs(temperatures - 4000)), :]

b = vega_mag(phoenix_wavelengths, k8v, trans_b, b_mstar, b_fstar)
v = vega_mag(phoenix_wavelengths, k8v, trans_v, v_mstar, v_fstar)
r = vega_mag(phoenix_wavelengths, k8v, trans_r, r_mstar, r_fstar)
i = vega_mag(phoenix_wavelengths, k8v, trans_r, r_mstar, i_fstar)

b - v, v - r, v - i

(1.1952683895991356, 0.70259301277640418, 1.4060508075475013)

Mamajek: K2V: 5040 K

B-V= 0.893, V-R = 0.487,  V-I = 0.929

In [125]:
k2v = model_fluxes[np.argmin(np.abs(temperatures - 5040)), :]

b = vega_mag(phoenix_wavelengths, k2v, trans_b, b_mstar, b_fstar)
v = vega_mag(phoenix_wavelengths, k2v, trans_v, v_mstar, v_fstar)
r = vega_mag(phoenix_wavelengths, k2v, trans_r, r_mstar, r_fstar)
i = vega_mag(phoenix_wavelengths, k2v, trans_r, r_mstar, i_fstar)

b - v, v - r, v - i

(0.86276810409825089, 0.52777963943150041, 1.2312374342025976)